In [27]:
import sqlite3
import pandas as pd
import numpy as np

In [45]:
#data
df = pd.read_excel('gcpdata.xlsx')

#country concordance 
country = pd.read_excel('GCP Country.xlsx')

df.set_index('Year', inplace=True)
df_transposing = df.transpose()
df_transposing.reset_index(inplace=True)
df_transposing.rename(columns={'index': 'Country GCP'}, inplace=True)

year_columns = [col for col in df_transposing.columns if str(col).isdigit()]
df_transposing[year_columns] /= 1_000
df_transposing['Earliest'] = df_transposing[year_columns].bfill(axis=1).iloc[:, 0]
df_transposing['MostRecent'] = df_transposing[year_columns].ffill(axis=1).iloc[:, -1]


#change decimal place according to the datadict
df_transposing[year_columns + ['Earliest', 'MostRecent']] = df_transposing[year_columns + ['Earliest', 'MostRecent']].round(7)
df_transposing= country.merge(df_transposing, on='Country GCP', how='left')
df_transposing = df_transposing.drop(columns='Country GCP')
df_transposing
#df_transposing.to_excel('gcpdatatransposed.xlsx', index=False)



C:\Users\Norah\AppData\Local\Temp\ipykernel_13068\2963037789.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_transposing['Earliest'] = df_transposing[year_columns].bfill(axis=1).iloc[:, 0]
C:\Users\Norah\AppData\Local\Temp\ipykernel_13068\2963037789.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_transposing['MostRecent'] = df_transposing[year_columns].ffill(axis=1).iloc[:, -1]


,Country,FIPS_CODE,1850,1851,1852,1853,1854,1855,1856,1857,...,2016,2017,2018,2019,2020,2021,2022,2023,Earliest,MostRecent
0,Afghanistan,AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.002431,0.002641,0.002893,0.002955,0.003168,0.002804,0.002881,0.003008,4.000000e-06,0.003008
1,Albania,ALB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.001264,0.001445,0.001336,0.001317,0.001286,0.001401,0.001412,0.001404,2.000000e-06,0.001404
2,Algeria,DZA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.043254,0.045080,0.047136,0.049101,0.045900,0.049016,0.050371,0.048617,0.000000e+00,0.048617
3,Angola,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.006950,0.006440,0.006361,0.006017,0.004512,0.005239,0.005527,0.005672,5.100000e-05,0.005672
4,Argentina,ARG,NaN,NaN,NaN,NaN,NaN,NaN,0.000005,0.000011,...,0.051834,0.051009,0.049290,0.048719,0.045066,0.052158,0.055699,0.053441,4.500000e-06,0.053441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,"Venezuela, Bolivarian Republic",VEN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.041679,0.037957,0.030356,0.028736,0.022087,0.021414,0.023885,0.027175,3.000000e-07,0.027175
184,Viet Nam,VNM,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.060311,0.062122,0.069683,0.092562,0.098925,0.085691,0.081232,0.091355,0.000000e+00,0.091355
185,Yemen,YEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.002731,0.002646,0.003165,0.003231,0.002971,0.002967,0.002810,0.002739,1.600000e-05,0.002739
186,Zambia,ZMB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.001600,0.001870,0.002025,0.002115,0.002223,0.002125,0.002053,0.002115,6.360000e-04,0.002115


In [46]:
#sql table; change table name each time you import 
conn = sqlite3.connect(r'C:\IFs\RUNFILES\IFsDataImport.db')
cursor = conn.cursor()
table_name = 'SeriesEmissionsCarbonCDIAC'
columns_sql = ['Country VARCHAR(255)', 'FIPS_CODE VARCHAR(255)']
columns_sql += [f'"{col}" DOUBLE(53)' for col in year_columns + ['Earliest', 'MostRecent']]
create_table_sql = f"CREATE TABLE '{table_name}' ({', '.join(columns_sql)})"
cursor.execute(f'DROP TABLE IF EXISTS "{table_name}"')
cursor.execute(create_table_sql)

df_transposing.to_sql(table_name, conn, if_exists='append', index=False)

conn.commit()
conn.close()